In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/home/farhanhubble/anaconda3/envs/dl-tflow-src/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X = np.load('combined.npy') 

In [3]:
X = (X.astype(np.float32)-127)/127.0

In [4]:
from tensorflow.core.framework import graph_pb2

In [5]:
graph_def = graph_pb2.GraphDef()

In [6]:
def load_graph():
    with open('./checkpoints/frozen_graph.pb', "rb") as f:
        graph_def.ParseFromString(f.read())
        with tf.Graph().as_default() as graph:
            input_big = tf.placeholder(dtype=tf.float32, shape=(None, 64,84,1), name='input_image_big')
            tf.import_graph_def(graph_def, name="", input_map={"reshaped_image": input_big})
            return graph

In [7]:
frozen_graph = load_graph()

In [8]:
for op in frozen_graph.get_operations():
    print(op.name)

input_image_big
input_image
reshaped_image/shape
reshaped_image
conv1/kernel
conv1/kernel/read
conv1/bias
conv1/bias/read
conv1/Conv2D
conv1/BiasAdd
conv1/Relu
pool1/MaxPool
conv2/kernel
conv2/kernel/read
conv2/bias
conv2/bias/read
conv2/Conv2D
conv2/BiasAdd
conv2/Relu
poool2/MaxPool
flatten/Shape
flatten/strided_slice/stack
flatten/strided_slice/stack_1
flatten/strided_slice/stack_2
flatten/strided_slice
flatten/Reshape/shape/1
flatten/Reshape/shape
flatten/Reshape
dense32/kernel
dense32/kernel/read
dense32/bias
dense32/bias/read
dense32/MatMul
dense32/BiasAdd
dense32/Relu
dense10/kernel
dense10/kernel/read
dense10/bias
dense10/bias/read
dense10/MatMul
dense10/BiasAdd
predictions


In [13]:
with tf.Session(graph=frozen_graph) as sess:
    # Get placeholder and output tensors.
    # Tensor names are derived from the operation that produced them
    # We named operations while building the graph NOT tensors.
    input_ph = tf.get_default_graph().get_tensor_by_name('input_image_big:0')
    bottleneck_tensor = tf.get_default_graph().get_tensor_by_name('poool2/MaxPool:0')
    
    # Select 5 random images
    indices = np.random.randint(0,len(X),[5])
    X_batch = X[indices]
    X_batch = np.expand_dims(X_batch,3)
    
    bottleneck_features = sess.run(bottleneck_tensor,feed_dict={input_ph:X_batch})
    print(bottleneck_features.shape)


(5, 14, 19, 8)


In [ ]:
for i in range(len(X)):
    plt.figure(figsize=(0.25,0.25))
    plt.imshow(X[i].reshape([28,28]),cmap='gray')
    plt.title('Predicted as {}'.format(y_predicted[i]))
    plt.show()